In [ ]:
import PyPDF2
import re
import os

# --- Configuración ---
PDF_PATH = "Manual_Empleado_Patio.pdf"
MIN_MATCH_SCORE = 2 # Requiere al menos 2 palabras clave para considerar una página relevante

# --- Funciones ---

def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de cada página de un archivo PDF.

    Args:
        pdf_path (str): La ruta al archivo PDF.

    Returns:
        list: Una lista donde cada elemento es el texto de una página.
              Devuelve una lista vacía si hay un error o el archivo no existe.
    """
    pages_text = []
    if not os.path.exists(pdf_path):
        print(f"Error: El archivo PDF '{pdf_path}' no fue encontrado.")
        return pages_text
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"Leyendo {num_pages} páginas del PDF...")
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                pages_text.append(page.extract_text() or "") # Añade string vacío si no extrae nada
            print("Extracción de texto completada.")
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
    return pages_text

def preprocess_text(text):
    """
    Limpia y normaliza el texto (minúsculas, elimina caracteres no alfanuméricos básicos).
    """
    text = text.lower()
    text = re.sub(r'[^\w\sáéíóúüñ]', '', text) # Mantiene letras acentuadas y ñ
    text = re.sub(r'\s+', ' ', text).strip() # Normaliza espacios
    return text

def find_relevant_page(query, pages_text):
    """
    Encuentra la página más relevante en el texto extraído basado en la consulta.
    Utiliza una puntuación simple basada en la coincidencia de palabras clave.

    Args:
        query (str): La pregunta del usuario.
        pages_text (list): Lista con el texto de cada página del PDF.

    Returns:
        str: El texto de la página más relevante o un mensaje de "no encontrado".
    """
    processed_query = preprocess_text(query)
    query_words = set(processed_query.split())

    if not query_words:
        return "Por favor, introduce una consulta válida."

    best_page_index = -1
    best_score = 0

    for i, page_content in enumerate(pages_text):
        processed_page = preprocess_text(page_content)
        page_words = set(processed_page.split())
        # Puntuación simple: número de palabras de la consulta presentes en la página
        current_score = len(query_words.intersection(page_words))

        # Da un bonus si palabras clave importantes están presentes (ejemplo)
        important_keywords = {"vacaciones", "licencia", "enfermedad", "salario", "despido", "ética", "confidencialidad", "horario"}
        if any(keyword in processed_page for keyword in important_keywords.intersection(query_words)):
             current_score += 1 # Pequeño bonus

        if current_score > best_score:
            best_score = current_score
            best_page_index = i

    if best_score >= MIN_MATCH_SCORE and best_page_index != -1:
        # Devuelve la página original (sin preprocesar) para legibilidad
        page_number = best_page_index + 1 # Número de página real
        return f"**Información encontrada (Página {page_number}, Puntuación: {best_score}):**\n\n{pages_text[best_page_index]}\n\n*Nota: Esta es la página completa que parece más relevante. Puede contener información adicional.*"
    else:
        return "Lo siento, no pude encontrar información específica sobre tu consulta en el Manual del Empleado. Intenta reformular tu pregunta o consulta directamente con Recursos Humanos."

# --- Función Principal ---

def main():
    """
    Función principal para ejecutar el chatbot.
    """
    print("Cargando Manual del Empleado...")
    manual_pages = extract_text_from_pdf(PDF_PATH)

    if not manual_pages:
        print("No se pudo cargar el manual. El chatbot no puede funcionar.")
        return

    print("\n--- Chatbot de Soporte RRHH (Basado en Manual) ---")
    print("Asociación Pro Desarrollo Comunal del Patio, Inc.")
    print("Escribe 'salir' para terminar.")

    while True:
        user_query = input("\n¿En qué puedo ayudarte hoy? > ")
        if user_query.lower() == 'salir':
            print("Gracias por usar el chatbot. ¡Hasta luego!")
            break

        response = find_relevant_page(user_query, manual_pages)
        print(f"\nRespuesta:\n{response}")

# --- Ejecución ---
if __name__ == "__main__":
    main()

Cargando Manual del Empleado...
Leyendo 29 páginas del PDF...
Extracción de texto completada.

--- Chatbot de Soporte RRHH (Basado en Manual) ---
Asociación Pro Desarrollo Comunal del Patio, Inc.
Escribe 'salir' para terminar.



¿En qué puedo ayudarte hoy? >  existen bonos?



Respuesta:
Lo siento, no pude encontrar información específica sobre tu consulta en el Manual del Empleado. Intenta reformular tu pregunta o consulta directamente con Recursos Humanos.



¿En qué puedo ayudarte hoy? >  ¿Hay bono de navidad?



Respuesta:
**Información encontrada (Página 3, Puntuación: 3):**

 
3 
Rev.1(07/09) TABLA DE CONTENIDO  
MISIÓN Y VISIÓN ............................................................................................................ 5  
BIENVENIDO (A) ............................................................................................................. 5  
ORGANIGRAMA .............................................................................................................. 7  
INFORMACION GENERAL ............................................................................................. 8  
Aplicabilidad del Manual ................................................................................................ 8  
POLITICAS Y PROCEDIMIENTOS ................................................................................ 8  
Igualdad de Oportuni dades de Empleo ........................................................................... 8  
Política Prohibiendo el Discrime n I


¿En qué puedo ayudarte hoy? >  Cuál es el código de ética?



Respuesta:
**Información encontrada (Página 17, Puntuación: 6):**

 
17 
Rev.1(07/09) Si es empleado regular y deja su empleo para ingresar en el servicio activo de las fuerzas 
armadas de los Estados Unidos, la Asociaci ón Pro Desarrollo Comunal del Patio, Inc. le 
reservará el empleo.  Deberá presentar al Director Ejecutivo, las órdenes oficiales de 
ingreso al servicio militar activo con por lo menos tres (3) semanas de anticipación.  Al 
regresar podrá reanudar la participación en  los planes de beneficios como si nunca 
hubiera estado afuera.  Al conclu ir la licencia para servicio militar tendrá para reportarse 
al Director Ejecutivo y solicitar la  reinstalación en el empleo: 
 
 6 meses – si estuvo en ejército militar 
 90 días – si fuese una licencia de más de 180 días pero, fu e ejército regular, 
 14 días – si la licencia fue desde 31 días hasta 180 días, 
 Si es una licencia de 30 días laborab les o menos, el empleado tiene hasta el 
próximo día laborable hábil lu ego 